In [1]:
import pandas as pd
import numpy as np
import random as rd
from numpy import log as ln

In [2]:

from collections import namedtuple


def load_problem(filename):
    """
    :rtype: object
    :param filename: The address to the problem input file
    :return: named tuple object of the problem attributes
    """
    A = []
    B = []
    C = []
    D = []
    E = []
    with open(filename) as f:
        lines = f.readlines()
        num_nodes = int(lines[1])
        num_vehicles = int(lines[3])
        num_calls = int(lines[num_vehicles + 5 + 1])

        for i in range(num_vehicles):
            A.append(lines[1 + 4 + i].split(','))

        for i in range(num_vehicles):
            B.append(lines[1 + 7 + num_vehicles + i].split(','))

        for i in range(num_calls):
            C.append(lines[1 + 8 + num_vehicles * 2 + i].split(','))

        for j in range(num_nodes * num_nodes * num_vehicles):
            D.append(lines[1 + 2 * num_vehicles + num_calls + 9 + j].split(','))

        for i in range(num_vehicles * num_calls):
            E.append(lines[1 + 1 + 2 * num_vehicles + num_calls + 10 + j + i].split(','))
        f.close()

    Cargo = np.array(C, dtype=np.double)[:, 1:]
    D = np.array(D, dtype=np.int)

    TravelTime = np.zeros((num_vehicles + 1, num_nodes + 1, num_nodes + 1))
    TravelCost = np.zeros((num_vehicles + 1, num_nodes + 1, num_nodes + 1))
    for j in range(len(D)):
        TravelTime[D[j, 0]][D[j, 1], D[j, 2]] = D[j, 3]
        TravelCost[D[j, 0]][D[j, 1], D[j, 2]] = D[j, 4]

    VesselCapacity = np.zeros(num_vehicles)
    StartingTime = np.zeros(num_vehicles)
    FirstTravelTime = np.zeros((num_vehicles, num_nodes))
    FirstTravelCost = np.zeros((num_vehicles, num_nodes))
    A = np.array(A, dtype=np.int)
    for i in range(num_vehicles):
        VesselCapacity[i] = A[i, 3]
        StartingTime[i] = A[i, 2]
        for j in range(num_nodes):
            FirstTravelTime[i, j] = TravelTime[i + 1, A[i, 1], j + 1] + A[i, 2]
            FirstTravelCost[i, j] = TravelCost[i + 1, A[i, 1], j + 1]
    TravelTime = TravelTime[1:, 1:, 1:]
    TravelCost = TravelCost[1:, 1:, 1:]
    VesselCargo = np.zeros((num_vehicles, num_calls + 1))
    B = np.array(B, dtype=object)
    for i in range(num_vehicles):
        VesselCargo[i, np.array(B[i][1:], dtype=np.int)] = 1
    VesselCargo = VesselCargo[:, 1:]

    LoadingTime = np.zeros((num_vehicles + 1, num_calls + 1))
    UnloadingTime = np.zeros((num_vehicles + 1, num_calls + 1))
    PortCost = np.zeros((num_vehicles + 1, num_calls + 1))
    E = np.array(E, dtype=np.int)
    for i in range(num_vehicles * num_calls):
        LoadingTime[E[i, 0], E[i, 1]] = E[i, 2]
        UnloadingTime[E[i, 0], E[i, 1]] = E[i, 4]
        PortCost[E[i, 0], E[i, 1]] = E[i, 5] + E[i, 3]

    LoadingTime = LoadingTime[1:, 1:]
    UnloadingTime = UnloadingTime[1:, 1:]
    PortCost = PortCost[1:, 1:]
    output = {
        'n_nodes': num_nodes,
        'n_vehicles': num_vehicles,
        'n_calls': num_calls,
        'Cargo': Cargo,
        'TravelTime': TravelTime,
        'FirstTravelTime': FirstTravelTime,
        'VesselCapacity': VesselCapacity,
        'LoadingTime': LoadingTime,
        'UnloadingTime': UnloadingTime,
        'VesselCargo': VesselCargo,
        'TravelCost': TravelCost,
        'FirstTravelCost': FirstTravelCost,
        'PortCost': PortCost
    }
    return output


def feasibility_check(solution, problem):
    """
    :rtype: tuple
    :param solution: The input solution of order of calls for each vehicle to the problem
    :param problem: The pickup and delivery problem object
    :return: whether the problem is feasible and the reason for probable infeasibility
    """
    num_vehicles = problem['n_vehicles']
    Cargo = problem['Cargo']
    TravelTime = problem['TravelTime']
    FirstTravelTime = problem['FirstTravelTime']
    VesselCapacity = problem['VesselCapacity']
    LoadingTime = problem['LoadingTime']
    UnloadingTime = problem['UnloadingTime']
    VesselCargo = problem['VesselCargo']
    solution = np.append(solution, [0])
    ZeroIndex = np.array(np.where(solution == 0)[0], dtype=int)
    feasibility = True
    tempidx = 0
    c = 'Feasible'
    for i in range(num_vehicles):
        currentVPlan = solution[tempidx:ZeroIndex[i]]
        currentVPlan = currentVPlan - 1
        NoDoubleCallOnVehicle = len(currentVPlan)
        tempidx = ZeroIndex[i] + 1
        if NoDoubleCallOnVehicle > 0:

            if not np.all(VesselCargo[i, currentVPlan]):
                feasibility = False
                c = 'incompatible vessel and cargo'
                break
            else:
                LoadSize = 0
                currentTime = 0
                sortRout = np.sort(currentVPlan, kind='mergesort')
                I = np.argsort(currentVPlan, kind='mergesort')
                Indx = np.argsort(I, kind='mergesort')
                LoadSize -= Cargo[sortRout, 2]
                LoadSize[::2] = Cargo[sortRout[::2], 2]
                LoadSize = LoadSize[Indx]
                if np.any(VesselCapacity[i] - np.cumsum(LoadSize) < 0):
                    feasibility = False
                    c = 'Capacity exceeded'
                    break
                Timewindows = np.zeros((2, NoDoubleCallOnVehicle))
                Timewindows[0] = Cargo[sortRout, 6]
                Timewindows[0, ::2] = Cargo[sortRout[::2], 4]
                Timewindows[1] = Cargo[sortRout, 7]
                Timewindows[1, ::2] = Cargo[sortRout[::2], 5]

                Timewindows = Timewindows[:, Indx]

                PortIndex = Cargo[sortRout, 1].astype(int)
                PortIndex[::2] = Cargo[sortRout[::2], 0]
                PortIndex = PortIndex[Indx] - 1

                LU_Time = UnloadingTime[i, sortRout]
                LU_Time[::2] = LoadingTime[i, sortRout[::2]]
                LU_Time = LU_Time[Indx]
                Diag = TravelTime[i, PortIndex[:-1], PortIndex[1:]]
                FirstVisitTime = FirstTravelTime[i, int(Cargo[currentVPlan[0], 0] - 1)]

                RouteTravelTime = np.hstack((FirstVisitTime, Diag.flatten()))

                ArriveTime = np.zeros(NoDoubleCallOnVehicle)
                for j in range(NoDoubleCallOnVehicle):
                    ArriveTime[j] = np.max((currentTime + RouteTravelTime[j], Timewindows[0, j]))
                    if ArriveTime[j] > Timewindows[1, j]:
                        feasibility = False
                        c = 'Time window exceeded at call {}'.format(j)
                        break
                    currentTime = ArriveTime[j] + LU_Time[j]

    return feasibility, c


def cost_function(Solution, problem):
    """
    :param Solution: the proposed solution for the order of calls in each vehicle
    :param problem:
    :return:
    """

    num_vehicles = problem['n_vehicles']
    Cargo = problem['Cargo']
    TravelCost = problem['TravelCost']
    FirstTravelCost = problem['FirstTravelCost']
    PortCost = problem['PortCost']


    NotTransportCost = 0
    RouteTravelCost = np.zeros(num_vehicles)
    CostInPorts = np.zeros(num_vehicles)

    Solution = np.append(Solution, [0])
    ZeroIndex = np.array(np.where(Solution == 0)[0], dtype=int)
    tempidx = 0

    for i in range(num_vehicles + 1):
        currentVPlan = Solution[tempidx:ZeroIndex[i]]
        currentVPlan = currentVPlan - 1
        NoDoubleCallOnVehicle = len(currentVPlan)
        tempidx = ZeroIndex[i] + 1

        if i == num_vehicles:
            NotTransportCost = np.sum(Cargo[currentVPlan, 3]) / 2
        else:
            if NoDoubleCallOnVehicle > 0:
                sortRout = np.sort(currentVPlan, kind='mergesort')
                I = np.argsort(currentVPlan, kind='mergesort')
                Indx = np.argsort(I, kind='mergesort')

                PortIndex = Cargo[sortRout, 1].astype(int)
                PortIndex[::2] = Cargo[sortRout[::2], 0]
                PortIndex = PortIndex[Indx] - 1

                Diag = TravelCost[i, PortIndex[:-1], PortIndex[1:]]

                FirstVisitCost = FirstTravelCost[i, int(Cargo[currentVPlan[0], 0] - 1)]
                RouteTravelCost[i] = np.sum(np.hstack((FirstVisitCost, Diag.flatten())))
                CostInPorts[i] = np.sum(PortCost[i, currentVPlan]) / 2

    TotalCost = NotTransportCost + sum(RouteTravelCost) + sum(CostInPorts)
    return TotalCost

In [3]:
call1, call2, call3, call4, call5, call6 = load_problem("Call_7_Vehicle_3.txt"), load_problem("Call_18_Vehicle_5.txt"),load_problem("Call_35_Vehicle_7.txt"), load_problem("Call_80_Vehicle_20.txt"), load_problem("Call_130_Vehicle_40.txt"), load_problem("Call_300_Vehicle_90.txt")

all_calls = [call1, call2, call3, call4, call5, call6]

/var/folders/dr/3007thwd3wgdfvqwgn0g4jjw0000gn/T/ipykernel_86536/1697434818.py:38: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  D = np.array(D, dtype=np.int)
/var/folders/dr/3007thwd3wgdfvqwgn0g4jjw0000gn/T/ipykernel_86536/1697434818.py:50: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

In [4]:
prob = call1

sol = [0, 2, 2, 0, 1, 5, 5, 3, 1, 3, 0, 7, 4, 6, 7, 4, 6]



print(prob.keys())

feasiblity, c = feasibility_check(sol, prob)

Cost = cost_function(sol, prob)

print(feasiblity)
print(c)
print(Cost)

dict_keys(['n_nodes', 'n_vehicles', 'n_calls', 'Cargo', 'TravelTime', 'FirstTravelTime', 'VesselCapacity', 'LoadingTime', 'UnloadingTime', 'VesselCargo', 'TravelCost', 'FirstTravelCost', 'PortCost'])
True
Feasible
1871372.0


In [5]:
def dummy(prob):
    "Creating dummy variable where all calls are at the end"
    n = prob["n_vehicles"]
    C = prob["n_calls"]
    
    dummy = []
    
    for i in range(n):
        dummy.append(0)
    for i in range(1, C+1):
        dummy.append(i)
        dummy.append(i)
    return dummy
    

In [6]:
def get_change(current, previous):
    if current == previous:
        return 0.0
    try:
        return (abs(current - previous) / previous) * 100.0 
    except ZeroDivisionError:
        return 0

In [7]:

def flat(lst):
    for i, j in enumerate(lst):
        lst[i].append(0)
    lst = sum(lst, [])
    #print(lst)
    return lst
    
    



In [8]:
def vehicle_check(problem, solution, i, cost=False, time=False):
    """
    :rtype: tuple
    :param solution: The input solution of order of calls for each vehicle to the problem
    :param problem: The pickup and delivery problem object
    :param i: column of solution where desired vehicle sits
    :param cost: if True returns only cost of vehicle.
    :param time: if true returns only time of vehicle.
    :return: whether the problem is feasible and the reason for probable infeasibility
    """
    
    num_vehicles = problem['n_vehicles']
    Cargo = problem['Cargo']
    TravelTime = problem['TravelTime']
    FirstTravelTime = problem['FirstTravelTime']
    VesselCapacity = problem['VesselCapacity']
    LoadingTime = problem['LoadingTime']
    UnloadingTime = problem['UnloadingTime']
    VesselCargo = problem['VesselCargo']
    
              
    TravelCost = problem['TravelCost']
    FirstTravelCost = problem['FirstTravelCost']
    PortCost = problem['PortCost']
    #solution = np.append(solution, [0])
    #ZeroIndex = np.array(np.where(solution == 0)[0], dtype=int)
    currentVPlan = solution[i]
    feasibility = True
    tempidx = 0
    
    NotTransportCost = 0
    RouteTravelCost = np.zeros(num_vehicles)
    CostInPorts = np.zeros(num_vehicles)
    
    c = 'Feasible'
    
    
    currentTime = 0
    #currentVPlan = solution[tempidx:ZeroIndex[i]]
    #currentVPlan = currentVPlan - 1
    
    currentVPlan = list(map(lambda x: x - 1, currentVPlan))
    
    
    NoDoubleCallOnVehicle = len(currentVPlan)
    #tempidx = ZeroIndex[i] + 1
    if NoDoubleCallOnVehicle > 0:
        

        if not np.all(VesselCargo[i, currentVPlan]):
            feasibility = False
            c = 'incompatible vessel and cargo'
            #break
            
        else:
            LoadSize = 0
            #currentTime = 0
            sortRout = np.sort(currentVPlan, kind='mergesort')
            I = np.argsort(currentVPlan, kind='mergesort')
            Indx = np.argsort(I, kind='mergesort')
            LoadSize -= Cargo[sortRout, 2]
            LoadSize[::2] = Cargo[sortRout[::2], 2]
            LoadSize = LoadSize[Indx]
            if np.any(VesselCapacity[i] - np.cumsum(LoadSize) < 0):
                feasibility = False
                c = 'Capacity exceeded'
                #break
            Timewindows = np.zeros((2, NoDoubleCallOnVehicle))
            Timewindows[0] = Cargo[sortRout, 6]
            Timewindows[0, ::2] = Cargo[sortRout[::2], 4]
            Timewindows[1] = Cargo[sortRout, 7]
            Timewindows[1, ::2] = Cargo[sortRout[::2], 5]

            Timewindows = Timewindows[:, Indx]

            PortIndex = Cargo[sortRout, 1].astype(int)
            PortIndex[::2] = Cargo[sortRout[::2], 0]
            PortIndex = PortIndex[Indx] - 1

            LU_Time = UnloadingTime[i, sortRout]
            LU_Time[::2] = LoadingTime[i, sortRout[::2]]
            LU_Time = LU_Time[Indx]
            Diag = TravelTime[i, PortIndex[:-1], PortIndex[1:]]
            FirstVisitTime = FirstTravelTime[i, int(Cargo[currentVPlan[0], 0] - 1)]
            
            

            RouteTravelTime = np.hstack((FirstVisitTime, Diag.flatten()))
            
            FirstVisitCost = FirstTravelCost[i, int(Cargo[currentVPlan[0], 0] - 1)]
            RouteTravelCost[i] = np.sum(np.hstack((FirstVisitCost, Diag.flatten())))
            CostInPorts[i] = np.sum(PortCost[i, currentVPlan]) / 2

    

            ArriveTime = np.zeros(NoDoubleCallOnVehicle)
            for j in range(NoDoubleCallOnVehicle):
                ArriveTime[j] = np.max((currentTime + RouteTravelTime[j], Timewindows[0, j]))
                if ArriveTime[j] > Timewindows[1, j]:
                    feasibility = False
                    c = 'Time window exceeded at call {}'.format(j)
                    break
                currentTime = ArriveTime[j] + LU_Time[j]
      

    

    
        if i == num_vehicles:
            NotTransportCost = np.sum(Cargo[currentVPlan, 3]) / 2
    

           

    TotalCost = NotTransportCost + sum(RouteTravelCost) + sum(CostInPorts)
    
    if cost == True:
        return TotalCost
    
    if time == True:
        return currentTime
    
    return feasibility, c, TotalCost

In [9]:
def highest_cost(problem, vehicles):
    #print('highest cost')
    num_vehicles = problem['n_vehicles']
    vessels = [i for i in range (0, num_vehicles)]
    C = []
    
    solutions = []
    for i in vessels:
        C = vehicle_check( problem, vehicles, i, cost=True)
        solutions.append(C)
    
    sol = max(solutions)
    indx = solutions.index(sol)
    #print (sol, indx)
    return sol, indx
        

In [10]:
def fullest_capacity(problem, vehicles):
    #print('fullest capacity')
    num_vehicles = problem['n_vehicles']
    VesselCapacity = problem['VesselCapacity']
    dif = [(VesselCapacity[i] - vehicle_check(problem, vehicles, i, cost=True)) for i, j in enumerate(vehicles)]
    sol = max(dif)
    indx = dif.index(sol)
    #print (sol, indx)
    return sol, indx
    

In [11]:
def longest_route (problem, vehicles):
    #print('longest route')
    times = [vehicle_check(problem, vehicles, i, time=True) for i, j in enumerate(vehicles)]
    l = max(times)
    indx = times.index(l)
    #print (l, indx)
    return l, indx
    
    
    

    

In [12]:
def min_n_calls(problem, vehicles):
    lengths = [len(i) for i in vehicles]
    m = min(lengths)
    idx = lengths.index(m)
    #print('min n calls')
    return m, idx

In [13]:
def randomness(problem, vehices):
    n_vehicles = problem['n_vehicles']
    idx = rd.randint(0,n_vehicles-1)
    return n_vehicles, idx

In [14]:
import timeit

In [15]:
functions = {1: fullest_capacity, 2: highest_cost, 3: longest_route, 4: min_n_calls, 5:randomness}


In [16]:
import copy

In [17]:
def NotTransportCostFunction(problem, undelivered):
    """
    Create a function that quickly calculates the cost of not transport for all the calls in undelivered-list
    """
    NotTransported  = 0
    Cargo = problem['Cargo']
    for i in undelivered:
        idx = i-1
        NotTransported += Cargo[idx][3]
    NotTransported  = NotTransported/2
    return NotTransported


In [18]:
def get_undelivered(problem, vehicles):
    n_calls = problem['n_calls']
    n_vehicles = problem['n_vehicles']
    
    
    undelivered = [i for i  in range(1,n_calls+1)]
    for i, j in enumerate(vehicles):
        for call in j:
            if call in undelivered:
                undelivered.remove(call)
    
    
    rd.shuffle(undelivered)
    
        
    return undelivered

In [19]:
def roulette(solutions, costs, sol2=None, cost2=None):
    # Create a list of weights for each solution (the sum of weights should be 1)
    weights = [1/len(solutions) for i in solutions]

    # Create a cumulative probability distribution
    cumulative_prob = []
    cumulative = 0
    for weight in weights:
        cumulative += weight
        cumulative_prob.append(cumulative)

    # Generate a random number between 0 and 1
    rand_num = random.random()

    # Find the index of the selected solution using the cumulative probability distribution
    for i, prob in enumerate(cumulative_prob):
        if rand_num <= prob:
            if sol2 == None:
                selected_solution = solutions[i]
                selected_sol_cost = costs[i]
                break
            else:
                selected_solution = solutions[i]
                selected_sol_cost = costs[i]
                #selected_solution2 = sol2[i]
                #selected_sol_cost2 = cost2[i]
                break
    

    #print("Selected solution:", selected_solution, 'selected_sol_cost', selected_sol_cost)
    
    if sol2 == None:
        return selected_solution, selected_sol_cost
    else:
        return selected_solution, selected_sol_cost


In [20]:
def destination_origin(problem):
    cargo = problem['Cargo']
    pairs = []
    for n, call in enumerate(cargo):
        
        for n2, call2 in enumerate(cargo):
            
            
            
            con = (n != n2) & (call[0] == call2[1]) 
            if con.all():
                
                # destination to origin of next node
                
                pairs.append((n2+1,n+1))
    return pairs
                

In [22]:
def dict_of_do_pairs_and_vehicles(problem):
    vc = problem['VesselCargo']
    N = problem['n_vehicles']
    d = []
    do_pairs = destination_origin(problem)
    for i in range(N):
        d.append([])
    

    for n, v in enumerate(vc):
        
        for p,r in do_pairs:
            condition = (v[p-1] == v[r-1] == 1)
            if condition.all():
                #print(n+1, (p,r))
                d[n].append((p,r))
        
    dic = {}
    for v, k in enumerate(d):
        
        dic[v] = k
        
    return dic

In [23]:
def indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

In [24]:
def close_friends(problem, solution,i, do_pairs, undelivered):
    """
    param solution: current solution
    param i: index that we should apply operator on
    
    """
    new_solutions, costs = [], []
    vehicles = copy.deepcopy(solution)
    vehicle = vehicles[i]
    cf = do_pairs[i]
    p = False
    for pair in cf:
        variations = []
        
        c1, c2 = pair
        
        if c1 in undelivered and c2 in undelivered:
            variations.append([c1,c1,c2,c2]), variations.append([c2,c2,c1,c1])
            
        elif c1 in vehicle:
            print('c1 in vehicle')
            p = True
            
            v = copy.deepcopy(vehicle)
            idx_c1 = indices(v, c1)[1]
            v.insert(idx_c1+1, c2), v.insert(idx_c1+1, c2)
            faux_solution = copy.deepcopy(vehicles)
            faux_solution[i] = v
            
            feasibility, c, cost = vehicle_check(problem, faux_solution, i)
            
            if feasibility and c == 'Feasible':
                print(v)
                new_solutions.append(v), costs.append(cost)
                
        elif c2 in vehicle:
            print('c2 in vehicle')
            p = True
            v = copy.deepcopy(vehicle)
            idx_c2 = indices(v, c2)[0]
            
            if idx_c2 == 0:
                v.insert(idx_c2, c1), v.insert(idx_c2, c1)
            else:
            
                v.insert(idx_c2-1, c1), v.insert(idx_c2-1, c1)
            faux_solution = copy.deepcopy(vehicles)
            faux_solution[i] = v
            
            feasibility, c, cost = vehicle_check(problem, faux_solution, i)
            if feasibility and c == 'Feasible':
                print(v)
                new_solutions.append(v), costs.append(cost)
            
        
            
        
        if len(vehicle) == 0 and len(variations) != 0:
            v = copy.deepcopy(vehicle)
            
            for l in variations:
                v = l
                vehicles[i] = v
                print(v)
                feasibility, c, cost = vehicle_check(problem, vehicles, i)
                if feasibility == True and c == 'Feasibile':
                    print(v)
                    new_solutions.append(v), costs.append(cost)
        else:
            
            
            for idx in range(len(vehicle) ):
                v = copy.deepcopy(vehicle)
                if len(variations) != 0:
            
                    for l in variations:
                        v[idx:idx] = l
                        vehicles[i] = v
                        print(v)
                        feasibility, c, cost = vehicle_check(problem, vehicles, i)
                        if feasibility == True and c == 'Feasibile':
                            print(v)
                            new_solutions.append(v), costs.append(cost)
                            
        
                    
    return new_solutions, costs
                    
        
        
    
    
    

In [25]:
vessels = [[], [3,3],[]]

In [1223]:
def node_call(problem):
    """
    returns two lists: 
    node_call_or: tuple of node, and call with that node as origin
    node_call_des: tuple of node, and call with that node as destination
    """
    nodes = problem["n_nodes"]
    cargo = problem["Cargo"]
    node_call_or = []
    node_call_des = []
    for n in range(nodes):
        for idx, call in enumerate(cargo):
            if call[0] == n:
                pair = (n, idx+1)
                node_call_or.append(pair)
            elif call[1] == n:
                
                pair = (n, idx+1)
                node_call_des.append(pair)
        
    return node_call_or, node_call_des

In [1226]:
def dict_of_nodecalls(problem):  
    vc = problem['VesselCargo']
    N = problem['n_vehicles']
    
    node_call_origin, node_call_des = node_call(problem)
    d1, d2 = [], []
    for i in range(N):
        d1.append([])
        d2.append([])
    for n, v in enumerate(vc):
        for node, call in node_call_origin:
            if v[call-1] == 1:
                
                d1[n].append((node, call))
    
    dic1 = {}
    for v, k in enumerate(d1):
        dic1[v] = k
        
    for n, v in enumerate(vc):
        for node, call in node_call_des:
            if v[call-1] == 1:
                
                d2[n].append((node, call))
    dic2 = {}
    for v, k in enumerate(d2):
        dic2[v] = k
        
    return dic1, dic2

In [1227]:
def get_size(problem, call):
    idx = call-1
    size = problem["Cargo"][idx][2]
    return size
get_size(call1, 1)

1886.0

In [1228]:
def size_chart(problem):
    N = problem["n_calls"]
    size_chart = {}
    for i in range(N):
        size_chart[i+1] = get_size(problem, i+1)
    return size_chart

In [1229]:
size_chart(call1)

{1: 1886.0,
 2: 11587.0,
 3: 5316.0,
 4: 8705.0,
 5: 10239.0,
 6: 14168.0,
 7: 10228.0}

In [1230]:
def heavy_luggage(problem, solution, idx):
    capacity = problem["VesselCapacity"][idx]
    vehicle = solution[idx]
    calls = []
    size_indx = []
    for indx, c in enumerate(vehicle):
        if c in calls:
            calls.remove(c)
        else:
            calls.append(c)
        size_indx = [capacity - (sum(get_size(problem, c) for c in calls)) ]
        
    return size_indx

        
        
    
        

In [1231]:
call1["VesselCargo"][2]

array([1., 1., 1., 0., 1., 1., 1.])

In [1232]:
def more_weight(problem, solution, hl, idx, idx2=None):
    
    vehicles = copy.deepcopy(solution)
    v = vehicles[idx]
    sc = size_chart(problem)
    vc = problem["VesselCargo"][idx]
    fs_calls = []
    new_sols, costs = [], []
    for i, j in enumerate(vc):
        if j == 1:
            fs_calls.append(i+1)
        else:
            sc.pop(i+1)
            
    undelivered = get_undelivered(problem, solution)
    

    for i in hl:
        for j in sc:
            if sc[j] < hl[i]:
                v1 = copy.deepcopy(v)
                if j in undelivered:
                
                    v1.insert(i+1, j)
                    vehicles[idx] = v1
                    f, c, cost = vehicle_check(problem, vehicles, idx)
                #print(v1,c)
                    if f and c == "Feasible":
                        l = len(v1)
                        for r in range(i, l):
                            v2 = copy.deepcopy(v1)
                            v2.insert(r,j)
                            vehicles[idx] = v2
                            f, c, cost = vehicle_check(problem, vehicles, idx)
                            #print(v2,c)
                            if f and c == "Feasible":
                                new_sols.append(v2), costs.append(cost)
                            
    return new_sols, costs

                

In [1233]:
def time_limit(problem, solution, idx, nodes=False):
    """
    Returns a dictionary of time passed in each index of vehicle, and a list of each node the vehicle visits.
    Same node twice in a row means the vehicle occupies itself with two calls on the same node
    """
    loading_time = problem["LoadingTime"][idx]
    unloading_time = problem["UnloadingTime"][idx]
    traveltime =  problem["TravelTime"][idx]
    first_tt = problem["FirstTravelTime"][idx]
    cargo = problem["Cargo"]
    
    vehicle = solution[idx]
    calls = []
    time_passed = []
    nodes = []
    for indx, c in enumerate(vehicle):
        lt, ut = 0, 0
        
        
        if c not in calls:
            calls.append(c)
            node = int(cargo[c-1][0])
            nodes.append(node)
            lt = loading_time[c-1]
            
        elif c in calls:
            calls.append(c)
            node = int(cargo[c-1][1])
            nodes.append(node)
            ut = unloading_time[c-1]
            
        #print("node", node)
        if indx == 0:
            ftt = first_tt[node-1]
            time_passed.append(ftt + lt)
        else:
            prev_call = vehicle[indx-1]
            if calls.count(prev_call) == 2:
                
                prev_node = nodes[indx-1]
                
            else:
                
                prev_node = nodes[indx-1]
            tt = traveltime[prev_node-1][node-1]
            time_passed.append(time_passed[indx-1] + (tt + lt + ut))
    if nodes == True:
        
        return time_passed, nodes
    else:
        return time_passed
                
                
            
        
            
            
        

In [1235]:

from itertools import permutations
def generate_feasible_shuffles(problem, solution, idx):
    """
    Generates all feasible shuffles of a list of calls
    """
    #n_calls = len(solution) // 2
    
    feasible_shuffles, times = [], []
    faux_solution = copy.deepcopy(solution)
    calls = copy.deepcopy(solution[idx])
    N = len(calls)
    perm_done = []
    for perm in permutations(calls):
        if perm in perm_done:
            continue
        else:
            perm_done.append(perm)
        
        faux_solution[idx] = perm
        
        feas, c, cost = vehicle_check(problem, faux_solution, idx)
        if feas and c == "Feasible":
            
            feasible_shuffles.append(list(perm))
            time = time_limit(problem, faux_solution, idx)
            times.append(time[N-1])
    
    
    fastest = min(times)
    
    fastest_idx = times.index(fastest)
    new_sol = feasible_shuffles[fastest_idx]
        
    return new_sol

        

In [1236]:
vessels = [[7,7], [5,3,5,3],[6,6]]

In [1237]:
def make_time_efficient(problem, solution):
    for idx, sol in enumerate(solution):
        ns = generate_feasible_shuffles(problem, solution, idx)
        solution[idx] = ns
        

In [931]:
def distance(problem, idx, node, node2):
    traveltime = problem["TravelTime"][idx]
    return traveltime[node-1][node2-1]
    

In [1240]:
def around_the_corner(problem, solution, info, idx, idx2):
    time, nodes = time_limit(problem,solution, idx)
    time2, nodes2 = time_limit(problem, solution, idx2)
    distances, indeces = [], []
    for node in nodes:
        distances.append([])
        indeces.append([])
    for indx, node in enumerate(nodes):
        for indx2, n2 in enumerate(nodes2):
            dist = distance(problem, idx, node, n2)
            #print(dist)
            distances[indx].append(dist)
            indeces[indx].append((indx,indx2))
    print(distances)
    sorted_distances = sorted ((value, index) for index, value in enumerate(distances))
    x = sorted_distances[0][1]
    print(indeces[x])
    if vessels[idx2][0]:
        print("k")
            
    
    

In [1243]:
def same_origin (problem):
    d = []
    for idx, c in enumerate(problem['Cargo']):
        for idx2, c2 in enumerate(problem['Cargo']):
        
            if c[0] == c2[0] and idx != idx2:
                d.append((idx+1, idx2+1))
    return d
        
        
#call2['TravelTime'][0][14]

In [1247]:
def relatedness(problem, c1, c2):
    """
    calculating relatedness between two calls.
    param: a,b,c,d 
    param: dist_p: average distance between pickup nodes for each vehicle
    param: dist_d: average distance between delivery nodes for each vehicle
    param: Ku, Ki, Kj: count of how many vehicles the calls are eligible for. U for both, i for c1, j for c2
    param:lowerbound_p: lowerbound for pickup
    param: upperbound_d: upperbound for delivery
    param: Size: difference in size between calls
    """
    a, b, c, d = 3, 1, 1, 50
    N = problem["n_vehicles"]
    cargo = problem["Cargo"]
    vc = problem["VesselCargo"]
    np1, np2 = int(cargo[c1-1][0]), int(cargo[c2-1][0]) # nodes for pickup
    
    nd1, nd2 = int(cargo[c1-1][1]), int(cargo[c2-1][1]) # nodes for delivery
    dist_p, dist_d = 0, 0
    Ki, Kj, Ku = 0, 0, 0
    for i in range(N):
        
        dist_p += distance(problem, i, np1, np2)
        dist_d += distance(problem, i, nd1, nd2)
        
        if vc[i][c1-1] == 1:
            Ki += 1
        if vc[i][c2-1] == 1:
            Kj += 1
            if vc[i][c2-1] == vc[i][c1-1]:
                Ku += 1
    
        
    dist_p /= N
    dist_d /= N
    #print(dist_p, dist_d)
    
    K = 1 - Ku/min(Kj,Ki)
    
    lowerbound_p1, lowerbound_p2 = cargo[c1-1][4], cargo[c2-1][4]
    upperbound_d1, upperbound_d2 = cargo[c1-1][7], cargo[c2-1][7]
    Ta = abs(lowerbound_p1 - lowerbound_p2)
    Tb = abs(upperbound_d1 - upperbound_d2)
    T = Ta + Tb
    size_c1, size_c2 = sz[c1], sz[c2]
    Size = abs(size_c1-size_c2)
    
    related = a * (dist_p + dist_d) + b * T + c * Size + d * K
    
    return related

In [1248]:
def roulette_wheel(items):
    # Calculate the probability of selecting each item
    total_prob = sum(range(1, len(items)+1))
    item_probs = [(i+1)/total_prob for i in range(len(items))]

    # Set the probability of the first item to 0.3
    item_probs[0] = 0.6

    # Spin the roulette wheel
    spin = random.random()
    cum_prob = 0
    for i in range(len(items)):
        cum_prob += item_probs[i]
        if spin < cum_prob:
            return items[i]


In [1249]:
def shaw_removal(problem, solutions, q, p=relatedness):
    n_calls = problem["n_calls"]
    vehicles = copy.deepcopy(solutions)
    calls = [i+1 for i in range(n_calls)]
    und = get_undelivered(problem, vehicles)
    D = []
    for c in und:
        if c in calls:
            calls.remove(c)
                
        
                
    if len(calls) == 0:
        return und, vehicles
    req = rd.choice(calls)
    calls.remove(req)
    
    D.append(req)
    
    
    L = [(call, p(problem, req, call)) for call in calls]
    sorted_list = (sorted(L, key=lambda x: x[1]))
    #print(sorted_list)
    L = [i[0] for i in sorted_list]
    #print(L)
    
    while q > 0:
        if len(L) > 0:
            y = roulette_wheel(L)
            if y != None:
            #print(y, L)
                D.append(y), L.remove(y)
        
        q -= 1
        
    for vehicle in vehicles:
        for call in D:
            if call in vehicle:
                vehicle.remove(call), vehicle.remove(call)
    rd.shuffle(D)
    return D, vehicles



In [1526]:
def worst_removal(problem, solutions, q, index=None):
    n_vehicles = problem["n_vehicles"]
    vehicles = copy.deepcopy(solutions)
    costs_n_calls = []
    
    D = []
    if index == None:
        while q > 0:
            calls_done = []
            faux_vehicles = copy.deepcopy(vehicles)
            
            costs_n_calls = []
            idx = rd.randint(0, n_vehicles-1)
            
            v = faux_vehicles[idx]
            if len(v) == 0:
                q -= 1
                continue
            elif len(v) == 2:
                v = []
                vehicles[idx] = v
                continue
            for call in v:
                if call in calls_done:
                    continue
                calls_done.append(call)
                v1 = copy.deepcopy(v)
                
                v1.remove(call), v1.remove(call)
                faux_vehicles[idx] = v1
                cost = vehicle_check(problem, faux_vehicles, idx, cost=True)
                costs_n_calls.append((cost, call))
                
            if len(vehicles[idx]) > 2:
                sorted_list = (sorted(costs_n_calls, key=lambda x: x[0]))
                L = [i[1] for i in sorted_list]
                
                r = roulette_wheel(L)
                
                if r != None:
                    D.append(r), vehicles[idx].remove(r), vehicles[idx].remove(r)
                    #print(vehicles)
                
                
            
                
            
            q -= 1
            
           
    undelivered = get_undelivered(problem, vehicles)
    return undelivered, vehicles
            
    

In [1460]:
def random_removal(problem, solution, q):
    undelivered = get_undelivered(problem, solution)
    N = problem["n_calls"]
    vehicles = copy.deepcopy(solution)
    calls = [i+1 for i in range(N)]
    D = []
    for i in calls:
        if i in undelivered:
            calls.remove(i)
            
    while q > 0:
        if len(calls) > 0:
            x = rd.choice(calls)
            for v in vehicles:
                if x in v:
                    
                    v.remove(x), v.remove(x)
                    D.append(x)
        q-=1
    undelivered = get_undelivered(problem, vehicles)
    return undelivered, vehicles

In [2131]:
def greedy_insertion(problem, call, solution, current_costs):
    vc = problem["VesselCargo"]
    
    vehicles = copy.deepcopy(solution)
    vehicles1 = copy.deepcopy(solution)
    vehicle_costs = copy.deepcopy(current_costs)
    
    
        
        
    vehicles = copy.deepcopy(vehicles1)
    best_vehicle = None
    best_index = None
    best_cost = float("inf")
    best_time = float("inf")
    shuffled_vehicle = []
    feas_vehicles = [i for i, j in enumerate(vc) if j[call-1] == 1]
        
            
    rd.shuffle(feas_vehicles)
    
    
    for i in feas_vehicles:
        vehicle = vehicles[i]
        
        
        
            
            
            
        if len(vehicle) == 0:
            j, k = 0, 0
                
            faux_vehicles = copy.deepcopy(vehicles)
            v = faux_vehicles [i]
            v.insert(j,call), v.insert(k, call)
            feas, c, cost = vehicle_check(problem, faux_vehicles, i)
            if feas and c == "Feasible":
                if cost < best_cost:
                            best_cost = cost
                            best_index = (i,j,k)
                            
                                
                    
                
            
        for j in range(len(vehicle)+1):
            faux_vehicles = copy.deepcopy(vehicles)
            v = faux_vehicles[i]
            v.insert(j,call)
            feas, c, cost = vehicle_check(problem, faux_vehicles, i)
            if feas and c == "Feasible":
                for k in range(j, len(vehicle)+1):
                    v.insert(k, call)
                    feas, c, cost = vehicle_check(problem, faux_vehicles, i)
                    if feas and c == "Feasible":
                            
                        if cost < best_cost:
                            best_cost = cost
                            best_index = (i,j,k)
                            
                                
    
                    
                            
                        
        
        
                            
    if best_index == None:
        return vehicles1, vehicle_costs
                
    else:
            
        i, j, k = best_index
        new_sol = vehicles1[i]
        new_sol.insert(j, call), new_sol.insert(k, call)
        
        vc = vehicle_costs.copy()
        vc[i] = float(best_cost)
        
    #print(vehicles1)
    
    return vehicles1, vc
        
                
                            
                        
        
                    
                    
                

In [2165]:
def q_greedy_insertion(problem, calls, solution, current_cost, q):
    
    vehicles = copy.deepcopy(solution)
    vehicle_cost = copy.deepcopy(current_cost)
    calls = copy.deepcopy(calls)
    
    while q > 0:
        if len(calls) > 0:
            call = calls.pop(0)
            
        else:
            break
        vehicles, vehicle_cost = greedy_insertion(problem, call, vehicles, vehicle_cost)
        
        q -= 1
        
    return vehicles, vehicle_cost

In [2166]:
def get_portcost(problem, call, idx):
    pc = problem["PortCost"]
    return pc[idx][call-1]

In [1256]:
def get_callnotdeliveredcost(problem, call):
    cargo = problem["Cargo"]
    return cargo[call-1][3]

In [1258]:
def get_listnotdeliveredcost(problem, lst):
    s = 0
    while len(lst) > 0:
        call = lst.pop()
        s += get_callnotdeliveredcost(problem,call)
    return s

In [1775]:
def cci_operator(problem, call, solution, current_cost):
    """
    Insert call based on which vehicle has the fewest possible calls.
    Stops at first improvement to make running time lower
    """
    # Get the subset of vehicles that can handle the given call
    vesselcargo = problem["VesselCargo"]
    feasible_vehicles_idx = [i for i,j in enumerate(vesselcargo) if j[call-1]==1]
    feasible_vehicles = [(solution[i], i) for i in feasible_vehicles_idx]
    vx_score = [(i, sum(j)) for i, j in enumerate(vesselcargo) if j[call-1] == 1]
    vx_score = sorted(vx_score, key=lambda x: x[1])
    #print("vx score", vx_score)
    # Sort the feasible vehicles by their remaining capacity
    
    vehicles = copy.deepcopy(solution)

    # Try to insert the call into each feasible vehicle and select the best one
    best_vehicle = None
    best_cost = float('inf')
    size_call = get_size(problem, call)
    indeces_in_bv = (0, 0)
    vehicle_cost = current_cost
    
    

   


    for idx, score in vx_score:
        faux_solution = copy.deepcopy(vehicles)
        
        v = copy.deepcopy(faux_solution[idx])
        rc = heavy_luggage(problem, vehicles, idx)
        l = len(v)

        if l == 0:
            
            
            pos, pos2 = 0, 0
            faux_solution = copy.deepcopy(vehicles)
            v = faux_solution[idx]
            v.insert(pos, call), v.insert(pos2, call)
            feas, c, cost = vehicle_check(problem, faux_solution, idx)
            if feas and c == "Feasible":
                if cost - get_callnotdeliveredcost(problem, call) < best_cost:
                    best_cost = cost
                    best_vehicle = idx
                    indeces_in_bv = (pos, pos2)
                    break
                    #print("zero", best_cost, best_vehicle, v, indeces_in_bv)
        else:
            
            
            
            for pos in range(l+1):
            
                for pos2 in range(pos,  l+1):
                    
                    faux_solution = copy.deepcopy(vehicles)
                    v = faux_solution[idx]
                   
                    v.insert(pos, call)
                    v.insert(pos2, call)
                
                    
                    feas, c, cost = vehicle_check(problem , faux_solution, idx)
                    if feas and c == "Feasible":
                       
                        
                        if cost - get_callnotdeliveredcost(problem, call) < best_cost:
                            best_cost = cost
                            best_vehicle = idx
                            indeces_in_bv = (pos, pos2)
                            break
                            
                            
                            
    if best_vehicle is not None:
        vehicle = copy.deepcopy(vehicles[best_vehicle])
        pos, pos2 = indeces_in_bv
        vehicle.insert(pos, call), vehicle.insert(pos2, call)
        vehicles[best_vehicle] = vehicle
        
        vehicle_cost[best_vehicle] = float(best_cost)
        
    #print(vehicles, vehicle_cost)
    return vehicles, vehicle_cost
    

   


In [1261]:
def regret_insert(problem, call, solution, current_cost, k=1):
    # Get the subset of vehicles that can handle the given call
    #print("start",solution, current_cost)
    vesselcargo = problem["VesselCargo"]
    feasible_vehicles_idx = [i for i,j in enumerate(vesselcargo) if j[call-1]==1]
    feasible_vehicles = [(solution[i], i) for i in feasible_vehicles_idx]
    vehicles = copy.deepcopy(solution)
    vehicle_cost = copy.deepcopy(current_cost)

    # Sort the feasible vehicles by their remaining capacity
    feasible_vehicles.sort(key=lambda v: len(v))

    # Compute the cost savings for inserting the call between each pair of calls on each feasible vehicle
    cost_savings = []
    
    for vehicle, idx in feasible_vehicles:
        l = len(vehicle)
        if l == 0:
            i,j = 0, 0
            faux_solution = copy.deepcopy(vehicles)
            v = faux_solution[idx] 
            v.insert(i, call)
            v.insert(j, call)
            feas, c, cost = vehicle_check(problem, faux_solution, idx)
            if feas:
                cost_savings.append((idx, i, j, vehicle_cost[idx] - cost))
        else:
            
        
            for i in range(l+1):
                for j in range(i+1, l+2):
                    if j-i > 0:
                        faux_solution = copy.deepcopy(vehicles)
                        v = faux_solution[idx] 
                        v.insert(i, call)
                        v.insert(j, call)
                        #print(v)
                        feas, c, cost = vehicle_check(problem, faux_solution, idx)
                        if feas:
                            cost_savings.append((idx, i, j, vehicle_cost[idx] - cost - get_callnotdeliveredcost(problem, call)))
        

    # Sort the cost savings in decreasing order and select the top q feasible vehicle/call pairs
    cost_savings.sort(key=lambda x: x[3], reverse=True)
    k = min(k, len(cost_savings))
    top_k = cost_savings[:k]

    # Compute the cost of inserting the call and the resulting cost per vehicle for each of the q feasible vehicle/call pairs
    regret_values = []
    for idx, i, j, cost_saving in top_k:
        faux_solution = copy.deepcopy(vehicles)
        
        v1 = faux_solution[idx]
        
        v1.insert(i, call)
        v1.insert(j, call)
        
        feas, c, cost = vehicle_check(problem, faux_solution, idx)
        if feas:
            regret = vehicle_cost[idx] - cost - cost_saving
            regret_values.append((idx, i, j, regret, cost))

    # Select the feasible vehicle/call pair with the lowest regret and insert the call accordingly
    if len(regret_values) > 0:
        
        idx, i, j, regret, cost = min(regret_values, key=lambda x: x[3])
        vehicle = copy.deepcopy(vehicles[idx])
        vehicle.insert(i, call)
        vehicle.insert(j, call)
        vehicles[idx] = vehicle
        vehicle_cost[idx] = cost
    
    return vehicles, vehicle_cost 



In [1418]:
def regret_q_insert(problem, calls, solution, current_cost, q ):
    vehicles = copy.deepcopy(solution)
    vehicle_cost = copy.deepcopy(current_cost)
    calls = list(set(calls))
    while q > 0:
        if len(calls) > 0:
            call = calls.pop(0)
        else:
            break
        vehicles, vehicle_cost = regret_insert(problem, call, vehicles, vehicle_cost)
        q -= 1
    return vehicles, vehicle_cost

In [1271]:
def get_true_costs(problem, solution):
    undelivered = get_undelivered(problem, solution)
    undelivered_cost = get_listnotdeliveredcost(problem, undelivered)
    vehicles = copy.deepcopy(solution)
    vehicle_cost = []
    for i in range(len(vehicles)):
        vehicle_cost.append(vehicle_check(problem,vehicles, i, cost=True))
    s = sum(vehicle_cost) + undelivered_cost
    
    return s, vehicle_cost

    

In [1280]:
def costy(problem, solution):
    vehicles = copy.deepcopy(solution)
    u = get_undelivered(problem, vehicles)
    sol = flat(vehicles)
    for i in u:
        sol.append(i), sol.append(i)
    return cost_function(sol, problem)


In [1275]:
def get_true_costs(problem, solution):
    vehicle_cost = []
    for i in range(len(solution)):
        vehicle_cost.append(vehicle_check(problem,solution, i, cost=True))
    cost = costy(problem, solution)
    return cost, vehicle_cost
    

In [1417]:
def q_cci_insert(problem, calls, solution, current_cost, q,  ):
    """
    
    """
    
    
    vehicles = copy.deepcopy(solution)
    vehicle_cost = copy.deepcopy(current_cost)
    calls = list(set(calls))
    
    while q > 0:
        if len(calls) > 0:
            c = calls.pop(0)
        else:
            break
            #print("c", calls, c)
        vehicles, vehicles_cost = cci_operator(problem, c, vehicles, vehicle_cost)
        
        q -= 1
        
        
    return vehicles, vehicle_cost

In [1285]:
lst, vessels = shaw_removal(call1, vessels, 5)
#current_costs = [999999999 for i in range(len(vessels))]

undelivered = get_undelivered(call1, vessels)
rd.shuffle(undelivered)
undelivered

[3, 5, 1, 2, 7, 4, 6]

In [1286]:
lst = [6,6,1,1]

In [1288]:
relatedness(call2, 6, 8)

2607.6

In [1289]:
def dict_of_so_pairs(problem):
    vc = problem['VesselCargo']
    N = problem['n_vehicles']
    d = []
    so_pairs = same_origin(problem)
    for i in range(N):
        d.append([])
    

    for n, v in enumerate(vc):
        
        for p,r in so_pairs:
            print(p,r)
            condition = (v[p-1] == v[r-1] == 1)
            if condition.all():
                #print(n+1, (p,r))
                d[n].append((p,r))
        
    dic = {}
    for v in range(N):
        dic[v] = d[v]
        
    return dic


In [1290]:
def root(problem, solution, so_pairs, i, undelivered):
    """
    param solution: current solution
    param i: index that we should apply operator on
    
    """
    new_solutions, costs = [], []
    vehicles = copy.deepcopy(solution)
    
    vehicle = vehicles[i]
    cf = so_pairs[i]
    p = False
    for pair in cf:
        variations = []
        
        c1, c2 = pair
        
        if c1 in undelivered and c2 in undelivered:
            variations.append([c1,c2, c2, c1]), variations.append([c2, c1, c2, c1])
            variations.append([c1,c2, c1, c2]), variations.append([c1, c2, c2, c1])
            
            
            if len(vehicle) == 0:
                v = copy.deepcopy(vehicle)
            
                for l in variations:
                    v = l
                    vehicles[i] = v
                
                    feasibility, c, cost = vehicle_check(problem, vehicles, i)
                    
                    if feasibility == True and c == 'Feasible':
                        print(v, feasibility,c)
                        new_solutions.append(v), costs.append(cost)
                    
            elif len(vehicle) != 0:
            
            
                for idx in range(len(vehicle)):
                
                    
                    
            
                    for l in variations:
                        v = copy.deepcopy(vehicle)    
                        v[idx:idx] = l
                        if v in new_solutions:
                            continue
                        vehicles[i] = v
                        
                        feasibility, c, cost = vehicle_check(problem, vehicles, i)
                        
                        if feasibility == True and c == 'Feasible':
                            print(v, feasibility, c)
                            new_solutions.append(v), costs.append(cost)
                        else:
                            break
            
            
        elif c1 in vehicle and c2 in undelivered:
            print('c1 in vehicle')
            p = True
            faux_solution = copy.deepcopy(solution)
            
            idx_c1 = indices(vehicle, c1)[0]
            w = 0
            while w < 2:
                v = copy.deepcopy(vehicle)
                v.insert(idx_c1+w, c2)
                idx_end = len(v) - idx_c1
                faux_solution = copy.deepcopy(solution)
                if idx_end > 1:
                    for pos in range(idx_end):
                        
                        v1 = copy.deepcopy(v)
                        v1.insert(pos, c2)
                        if v1 in new_solutions:
                            continue
                        faux_solution[i] = v1
                    
                        feasibility, c, cost = vehicle_check(problem, faux_solution, i)
                        if feasibility and c == "Feasible":
                            print(v1)
                            new_solutions.append(v1), costs.append(cost)
                        else:
                            break
                w += 1
                    
    
            
            
    
                
        elif c2 in vehicle and c1 in undelivered:
            print('c2 in vehicle')
            p = True
            
            
            
            
            w = 0 
            while w < 2:
                v = copy.deepcopy(vehicle)
                idx_c2 = indices(v, c2)[0]
                v.insert(idx_c2+w, c1)
                idx_end = len(v) - idx_c2
                faux_solution = copy.deepcopy(solution)
                if idx_end > 1:
                    for pos in range(idx_end):
                        
                        v1 = copy.deepcopy(v)
                        v1.insert(pos, c1)
                        if v1 in new_solutions:
                            continue
                        faux_solution[i] = v1
                    
                        feasibility, c, cost = vehicle_check(problem, faux_solution, i)
                        if feasibility and c == "Feasible":
                            print(v1)
                            new_solutions.append(v1), costs.append(cost)
                        else:
                            break
                w += 1
                
                
                
                
            
        
            
        
        
                            
        
                    
    return new_solutions, costs

In [1291]:
def neighbourhood_root(problem, solution, so_pairs, idx, idx2=None):
    """
    Using the root operator.
    Removing up to two different calls to create diversification
    """
    vehicles = copy.deepcopy(solution)
    unde = get_undelivered(problem, vehicles)
    
    #print(do_pairs)
    u = get_undelivered(problem, vehicles)
    NT = NotTransportCostFunction(problem, u)
    new_sols, costs, diffs = [],[],[]
    new_sols[-1:-1], costs[-1:-1] = root(problem, vehicles, so_pairs, idx,unde )
    #print("new sols", new_sols)
    
    
    
    v = vehicles[idx]
    if len(v) > 0:
        call = rd.choice(v) 
        lngth = len(new_sols)
        v.remove(call), v.remove(call)
        vehicles[idx] = v
        new_sols[lngth:lngth], costs[lngth:lngth] = root(problem, vehicles, so_pairs, idx, unde )
    
    #print("new sols", new_sols)
    
    if len (v) > 0:
        unde = get_undelivered(problem, vehicles)
        call2 = rd.choice(v)
        v.remove(call2), v.remove(call2)
        vehicles[idx] = v
        lngth = len(new_sols)
        new_sols[lngth:lngth], costs[lngth:lngth] = root(problem, vehicles, so_pairs,idx,unde )
        print("new sols", new_sols)
        
    for sol in new_sols:
        
        vs = copy.deepcopy(solution)
        vs[idx] = sol
        
        new_und = get_undelivered(problem, vs)
        
        new_nt = NotTransportCostFunction(problem, new_und)
        delta_nt = NT - new_nt
        diffs.append(delta_nt)
        
    return new_sols, costs, diffs


In [1293]:
def check_undelivered(problem, solution, lst):
    undelivered = get_undelivered(problem,solution)
    for i in lst:
        if i in undelivered:
            continue
        if i not in undelivered:
            undelivered.append(i)
            for v in solution:
                if i in v:
                    v.remove(i), v.remove(i)

In [1838]:
def escape(problem, solution):
    vehicles = copy.deepcopy(solution)
    n_calls = problem["n_calls"]
    N = round(n_calls*0.5)
    d, sel_state = random_removal(problem, vehicles, N)
    cost, lst = get_true_costs(problem, sel_state)
    return sel_state, cost
    
    

In [1563]:
def update_weights(weights, scores, counts):
    avg_scores = [score / count for score, count in zip(scores, counts)]
    total_avg = sum(avg_scores) / len(avg_scores)
    new_weights = [total_avg / avg_score for avg_score in avg_scores]
    sum_weights = sum(new_weights)
    normalized_weights = [w / sum_weights for w in new_weights]
    return normalized_weights


In [1302]:
def roulette_wheel_weights(items, weights):
    """Select a random item from the list based on its weight using a roulette wheel selection algorithm."""
    total_weight = sum(weights)
    r = random.uniform(0, total_weight)
    cum_weight = 0
    for i, item in enumerate(items):
        cum_weight += weights[i]
        if cum_weight >= r:
            return item


In [2300]:
def simulated_annealing4(problem, cooling_rate=0.996):
    
    num_vehicles = problem['n_vehicles']
    n_calls = problem["n_calls"]
    warm_up_iterations, max_iterations =  100, 10000
    
    initial_state = dummy(problem)
    initial_cost = cost_function(initial_state, problem)
    
    current_state = make_into_vehicles(problem, initial_state)
    current_cost = get_true_costs(problem, current_state)
    
    best_state = copy.deepcopy(current_state)
    best_cost = copy.deepcopy(current_cost[0])
    current_cost_list = []
    sz = size_chart(problem)
    
    q_insert_init = round(n_calls*0.30)
    q_remove_init = round(n_calls*0.15)
    
    q = q_insert_init
    qr = min(2,q_remove_init)
    success = 5
    delta_ten = []
    
    warmup_delta = 0
    avg_delta = []
    
    lst = []
    q, qr = 4,2 
    ten = round(n_calls*0.10)
    
    list_of_operators = [q_cci_insert, regret_q_insert, q_greedy_insertion]
    list_of_removals = [shaw_removal, worst_removal, random_removal]
    operator_scores = [1 for i in list_of_operators]
    rem_scores = [1 for i in list_of_removals]
    weights = [1 for i in list_of_operators]
    rem_weights = [1 for i in list_of_removals]
    rem_counts = [1, 1, 1]
    
    solutions_explored = []
    
    current_state_count = 0
    counts = [1,1,1]
    best_state_count = 0
    
    for it in range(warm_up_iterations):
        
        
            
        
        
        vehicles = copy.deepcopy(current_state)
        current_cost, current_cost_list = get_true_costs(problem, vehicles)
        
        
        rx = random.random()
        if rx < 0.33:
        # Use the Shaw removal operator
            removal_op = shaw_removal
        elif rx < 0.66:
        # Use the random removal operator
            removal_op = random_removal
        else:
            removal_op = worst_removal
            
        
                
        # choose randomly between CCI operator and R-regret insert operator
        undelivered = get_undelivered(problem, vehicles)
        
        D, sel_state = removal_op(problem, vehicles, qr)
            
     
            
        
        lst = get_undelivered(problem, sel_state)
            
        
                
        # choose randomly between CCI operator and R-regret insert operator
        
        

        
        r = random.random()
        if r < 0.33:
            # apply CCI operator
            
            
            sel_state, sel_cost = q_cci_insert(problem, lst, sel_state, current_cost_list, q)
            
            
        elif r < 0.66:
            # apply R-regret insert operator
            
            sel_state, sel_cost = regret_q_insert(problem, lst, sel_state, current_cost_list, q)
            
        else:
            
            sel_state, sel_cost = q_greedy_insertion(problem, lst, sel_state, current_cost_list, q)
            
            # update best solution
            
    
        
        
        sel_cost, sel_cost_list = get_true_costs(problem, sel_state)
        
       
        
        delta_cost = sel_cost - current_cost  
            
        avg_delta.append(delta_cost)
                
        con = delta_cost < 0 & (random.random() < 0.80)
        if con.any():
            current_state, current_cost = copy.deepcopy(sel_state), copy.deepcopy(sel_cost)
                
        
        if (current_cost - best_cost) < 0:
            best_state = copy.deepcopy(current_state)
            best_cost = copy.deepcopy(current_cost)
            #print(best_state, best_cost)
            
                
        
    warm_up_delta = sum(avg_delta)/len(avg_delta)
    initial_temperature = - warm_up_delta/ln(0.8)
    temperature = abs(initial_temperature)
    #print("ini Temp")
    #print(initial_temperature)
    
                        
    threshold = 1000
    
    
            
    for it in range(warm_up_iterations, max_iterations):
        
        if it % 100 == 0:
            weights = update_weights(operator_scores, counts, weights)
            rem_weights = update_weights(rem_scores, rem_counts, rem_weights)
            counts = [1,1,1]
            rem_counts = [1, 1, 1]
            operator_scores = [1, 1, 1]
            rem_scores = [1, 1, 1]
        
        if it % 1000 == 0:
            print(it)
            #print(weights)
            #print(rem_weights)
             
            print(best_state, best_cost)
            if temperature > 500:
                print(math.exp(-(delta_cost)/temperature))
        
        if best_state_count > threshold:
            bs = copy.deepcopy(best_state)
            
            current_state, current_cost = escape(problem, bs)
            threshold += 500
            
            
            q = min(q,5)
            qr = min(qr,4)
            
        
            
        vehicles = copy.deepcopy(current_state)
        
        y = roulette_wheel_weights([0,1,2], rem_weights)
        removal_op = list_of_removals[y]
            
        
                
        # choose randomly between CCI operator and R-regret insert operator
        undelivered = get_undelivered(problem, vehicles)
        
        D, sel_state = removal_op(problem, vehicles, qr)
            
        if removal_op == shaw_removal:
            rd.shuffle(D)
            lst = D + undelivered
            
        else:
            lst = get_undelivered(problem, sel_state)
            
        
            
        r = random.random()
        #lst = list(set(lst))
        
        check_undelivered(problem, sel_state, lst)
        x = roulette_wheel_weights([0,1,2], weights)
        operator = list_of_operators[x]
        counts[x] += 1
        rem_counts[y] += 1
        
            
        sel_state, sel_cost = operator(problem, lst, sel_state, current_cost_list, q)
        sel_cost, sel_cost_list = get_true_costs(problem, sel_state)
            # update best solution
            
        
        
        delta_cost =  sel_cost - current_cost
        
        
       
            
        
        
        if delta_cost < 0:
            current_state, current_cost = copy.deepcopy(sel_state), copy.deepcopy(sel_cost)
            operator_scores[x] += 4
            rem_scores[y] += 4
            
        
        else:
            try:
                if (random.random() < math.exp(-(delta_cost)/temperature)):
                    current_state, current_cost = sel_state, sel_cost
                    
                
                    
            except OverflowError:
                if (random.random() < 0.8):
                    current_state, current_cost = sel_state, sel_cost
                    
                    
        
        
        
                
        
        current_cost, current_cost_list = get_true_costs(problem, current_state) 
        best_cost, best_cost_lost = get_true_costs(problem, best_state)
        
        if current_cost < best_cost:
            
            best_state = copy.deepcopy(current_state)
            best_cost = copy.deepcopy(current_cost)
            
            operator_scores[x] += 10
            rem_scores[y] += 10
            best_state_count = 0
            threshold = 1000
            
        else:
            best_state_count += 1
        
            
        
                
        
        # cool down
        temperature *= cooling_rate
        
        #sum_delta_ten = sum(delta_ten)
        
            
            
            
        
            
        
        #print("success", success)
        
    undelivered = get_undelivered(problem, best_state) 
    best_state = flat(best_state)
    for i in undelivered:
        best_state.append(i), best_state.append(i)
        
    best_cost = cost_function(best_state, problem)
    
    return best_state, best_cost

In [2301]:
sz = size_chart(call3)
rt = timeit.Timer(stmt=lambda: simulated_annealing4(call3)).repeat(repeat=1, number=1)

KeyboardInterrupt: 

In [2296]:
rt

[824.4909627499874]

In [2277]:
def models_final(problem):
    initial_cost = cost_function(dummy(problem), problem)
    
    models = [simulated_annealing4]
    sz = size_chart(problem)
    best_sols = []
    df = pd.DataFrame( columns=['Best Objective', 'Average Objective', 'Improvement (%)', 'Running Time'])
    
    for algorithm in models:
        sols, costs = [],[]
        
        for iteration in range(5):
            sol, cost = algorithm(problem)
            sols.append(sol), costs.append(cost)
            
        best_cost = min(costs)
        best_sol = sols[costs.index(best_cost)]
        best_sols.append(best_sol)
        print(best_sol)
        avg = sum(costs)/len(costs)
        imp = get_change(best_cost, initial_cost)
        rt = timeit.Timer(stmt=lambda: algorithm(problem)).repeat(repeat=2, number=1)
        rt = sum(rt)/len(rt)
        idx = models.index((algorithm))
        df.loc[idx] = [best_cost, avg, imp, rt]
        
    index_names = {0: "Final algorithm " 
                }
    df = df.rename(index=index_names)
        
    return df, best_sols
